In [6]:
import pandas as pd
import numpy as np
import os
import sys

# --- 0. 경로 및 설정 ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(PROJECT_ROOT, "processed")

# [입력 1] 이전 단계에서 저장한 마스터 파일
MASTER_FILE = os.path.join(DATA_DIR, "master_c_result.csv")
# [입력 2] IMO 이벤트 파일
IMO_FILE = os.path.join(DATA_DIR, "imo_all_with_regstage.csv")

# [출력] 12채널이 모두 완성된 최종 마스터 테이블
FINAL_MASTER_FILE = os.path.join(DATA_DIR, "final_master_table.csv")

print(f"Loading master file: {MASTER_FILE}")
print(f"Loading IMO file: {IMO_FILE}")

# --- 1. [Phase 1-C] 결과물 로드 ---
try:
    master_df = pd.read_csv(MASTER_FILE, parse_dates=['date'])
    # 'ticker'를 문자열로 명시 (병합 키 오류 방지)
    master_df['ticker'] = master_df['ticker'].astype(str)
except FileNotFoundError:
    print(f"\n[!!!] Error: {MASTER_FILE}을 찾을 수 없습니다.")
    sys.exit()

# --- 2. [Phase 1-B] IMO 이벤트 병합 ---
try:
    imo_df = pd.read_csv(IMO_FILE, parse_dates=['date_iso'])

    # [핵심 필터링] 'finalized_regulation'(최종 규제)만 사용
    imo_df_finalized = imo_df[imo_df['reg_stage'] == 'finalized_regulation'].copy()

    # 'imo_event_impulse' 채널 생성
    imo_df_finalized['imo_event_impulse'] = 1
    
    # 중복된 날짜의 이벤트는 하나로 합침 (하루에 여러 규제 발표 시 1로 통일)
    imo_df_finalized = imo_df_finalized.groupby('date_iso')['imo_event_impulse'].max().reset_index()
    
    # 'date_iso'를 'date'로 이름 변경하여 병합 준비
    imo_df_finalized = imo_df_finalized.rename(columns={'date_iso':'date'})

    # [병합] master_df에 'date' 기준으로 공통 이벤트 병합
    master_df = pd.merge(master_df, imo_df_finalized, on='date', how='left')
    
    # [NaN 처리] 이벤트가 없었던 날은 NaN이므로 0으로 채움
    master_df['imo_event_impulse'] = master_df['imo_event_impulse'].fillna(0)
    print("  > Merged IMO event data (impulse)")

except FileNotFoundError:
    print(f"\n[!!!] Error: {IMO_FILE}을 찾을 수 없습니다.")
    sys.exit()

# --- 3. [Phase 1-D] 최종 파생 변수 생성 ---
print("  > Creating final derived variables...")

# 3-1. 로그 변환 (log1p: 0이나 1을 안전하게 처리)
master_df['close_log'] = np.log1p(master_df['close'])
master_df['trading_volume_log'] = np.log1p(master_df['trading_volume'])

# 3-2. 일별 로그 수익률 (ret_1d)
# [중요] 반드시 groupby('ticker')로 기업별 계산
master_df = master_df.sort_values(by=['ticker', 'date']) # 정렬 필수
master_df['ret_1d'] = master_df.groupby('ticker')['close_log'].diff(1)

# 3-3. 수주 채널 (임시 생성, 0으로 채움)
print("  > [Note] Creating placeholder columns for 'Orders' (all zeros)")
master_df['new_order_amount_impulse'] = 0.0
master_df['new_order_amount_stair'] = 0.0

# 3-4. IMO 이벤트 감쇠 (Decay) 채널 (λ=0.95)
# (이 로직은 데이터가 클 경우 시간이 다소 걸릴 수 있습니다)

def calculate_decay(group, impulse_col, decay_factor=0.95):
    """
    그룹별(기업별)로 감쇠 채널을 계산하는 함수
    v_t = max(v_{t-1} * 0.95, impulse_t)
    """
    decay_series = pd.Series(0.0, index=group.index)
    last_decay_value = 0.0
    
    for i, (index, row) in enumerate(group.iterrows()):
        impulse = row[impulse_col]
        
        # 이벤트 당일은 1, 그 외는 0.95씩 감쇠
        current_decay = max(last_decay_value * decay_factor, impulse)
        
        decay_series.iloc[i] = current_decay
        last_decay_value = current_decay
        
    return decay_series

# groupby('ticker')로 기업별 감쇠 로직 적용
master_df['imo_event_decay'] = master_df.groupby('ticker', group_keys=False).apply(
    calculate_decay, 'imo_event_impulse'
)
print("  > Calculated IMO event decay channel")

Loading master file: /workspace/ship-ai/data/processed/master_c_result.csv
Loading IMO file: /workspace/ship-ai/data/processed/imo_all_with_regstage.csv
  > Merged IMO event data (impulse)
  > Creating final derived variables...
  > [Note] Creating placeholder columns for 'Orders' (all zeros)
  > Calculated IMO event decay channel


In [8]:
# --- 4. 최종 컬럼 선택 및 저장 ---

# [!!! 수정된 부분 !!!]
# 'roe_asof', 'real_debt_ratio_asof' -> 
# 실제 컬럼명인 'roe', 'real_debt_ratio'로 수정
FINAL_CHANNELS = [
    'date', 'ticker',
    'close_log',                # 1
    'ret_1d',                   # 2
    'trading_volume_log',       # 3
    'roe',                      # 4 (수정!)
    'real_debt_ratio',          # 5 (수정!)
    'new_order_amount_impulse', # 6 (임시 0)
    'new_order_amount_stair',   # 7 (임시 0)
    'bdi_proxy',                # 8 
    'wti',                      # 9 
    'newbuild_proxy_2015_100',  # 10
    'imo_event_impulse',        # 11
    'imo_event_decay'           # 12
]

# 원본 데이터(open, high, low 등)는 제외하고 최종 채널만 선택
final_master_table = master_df[FINAL_CHANNELS].copy()

# 초기 데이터(NaN) 제거
final_master_table = final_master_table.dropna()

# [저장] 12채널 마스터 테이블 저장
final_master_table.to_csv(FINAL_MASTER_FILE, index=False)

print(f"\n[SUCCESS] Phase 1 (Data Layer) 완료!")
print(f"12채널이 모두 포함된 'final_master_table.csv'을 저장했습니다.")
print(f"  -> {FINAL_MASTER_FILE}")

print("\n--- Final Table Info ---")
final_master_table.info()


[SUCCESS] Phase 1 (Data Layer) 완료!
12채널이 모두 포함된 'final_master_table.csv'을 저장했습니다.
  -> /workspace/ship-ai/data/processed/final_master_table.csv

--- Final Table Info ---
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5336 entries, 1566 to 1476
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   date                      5336 non-null   datetime64[ns]
 1   ticker                    5336 non-null   object        
 2   close_log                 5336 non-null   float64       
 3   ret_1d                    5336 non-null   float64       
 4   trading_volume_log        5336 non-null   float64       
 5   roe                       5336 non-null   float64       
 6   real_debt_ratio           5336 non-null   float64       
 7   new_order_amount_impulse  5336 non-null   float64       
 8   new_order_amount_stair    5336 non-null   float64       
 9   bdi_proxy                 5336 n